In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_train_path = "./../data/01_raw/yellow_tripdata_2022-01.parquet"
df_val_path = "./../data/01_raw/yellow_tripdata_2022-02.parquet"

In [3]:
df_train = pd.read_parquet(df_train_path)
df_val = pd.read_parquet(df_val_path)

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [6]:
def calc_duration(df_in: pd.DataFrame, dropoff_col: str, pickup_col: str, target_col: str):
    
    df_out = df_in.copy()
    df_out[target_col] = df_out[dropoff_col] - df_out[pickup_col]
    df_out[target_col] = df_out[target_col].dt.total_seconds()/60.0
    
    return df_out    

In [7]:
target_col = "duration"
dropoff_col = "tpep_dropoff_datetime"
pickup_col = "tpep_pickup_datetime"

In [8]:
df_train = calc_duration(df_train, dropoff_col, pickup_col, target_col)

In [9]:
df_train[[target_col]].describe()

,duration
count,2.463931e+06
mean,1.421220e+01
std,4.644531e+01
min,-3.442400e+03
25%,6.316667e+00
50%,1.018333e+01
75%,1.616667e+01
max,8.513183e+03


In [10]:
ii = (df_train[target_col] >= 1.0) & (df_train[target_col] <= 60.0)
ii.sum()/len(df_train)

0.9827547930522406

In [11]:
df_train = df_train[ii]

In [12]:
categorical = ['PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)

In [13]:
dict_train = df_train[categorical].to_dict(orient="records")
dict_train

[{'PULocationID': '142', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '42'},
 {'PULocationID': '166', 'DOLocationID': '166'},
 {'PULocationID': '114', 'DOLocationID': '68'},
 {'PULocationID': '68', 'DOLocationID': '163'},
 {'PULocationID': '138', 'DOLocationID': '161'},
 {'PULocationID': '233', 'DOLocationID': '87'},
 {'PULocationID': '238', 'DOLocationID': '152'},
 {'PULocationID': '166', 'DOLocationID': '236'},
 {'PULocationID': '236', 'DOLocationID': '141'},
 {'PULocationID': '141', 'DOLocationID': '229'},
 {'PULocationID': '114', 'DOLocationID': '90'},
 {'PULocationID': '234', 'DOLocationID': '113'},
 {'PULocationID': '246', 'DOLocationID': '79'},
 {'PULocationID': '43', 'DOLocationID': '140'},
 {'PULocationID': '239', 'DOLocationID': '151'},
 {'PULocationID': '148', 'DOLocationID': '141'},
 {'PULocationID': '237', 'DOLocationID': '107'},
 {'PULocationID': '7', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '263'},
 {'PULocationID': '263', 'DOLoc

In [14]:
dv = DictVectorizer()
X_train = dv.fit_transform(dict_train)

In [15]:
X_train.shape

(2421440, 515)

In [16]:
y_train = df_train[target_col]

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190135975749

In [18]:
df_val = calc_duration(df_val, dropoff_col, pickup_col, target_col)

In [19]:
df_val[categorical] = df_val[categorical].astype(str)
dict_val = df_val[categorical].to_dict(orient="records")
X_val = dv.transform(dict_val)

In [20]:
y_val = df_val[target_col]
y_pred = lr.predict(X_val)

In [21]:
mean_squared_error(y_val, y_pred, squared=False)

46.877266034389095